# [Std 04] - calculate Christoffel symbols in spherical coords

In [1]:
%display latex
import numpy as np
import scipy
from numpy import r_

from sage.manifolds.operators import *

First, let's define a Euclidean space in spherical coordinates, over the variables $r$, $\theta$, and $\phi$.  

In [2]:
M.<r, θ, φ> = EuclideanSpace(coordinates='spherical')
M.spherical_coordinates().coord_range()

r: (0, +oo); θ: (0, pi); φ: [0, 2*pi] (periodic)

To be complete, let's also check out its frames.  These are in order of its coordinate frame (a tuple of its basis and partials), it's basis frame (in spherical coordinates), which is derived as a Chart from the Cartesian coordinates.  Coordinate frames matter in GR, we will address that later. 

In [3]:
M.frames()

[Coordinate frame (E^3, (∂/∂r,∂/∂θ,∂/∂φ)), Vector frame (E^3, (e_r,e_θ,e_φ))]

* c.ref. https://math.stackexchange.com/q/1985964 and https://mathworld.wolfram.com/SphericalCoordinates.html; order of the parameter definitions matter!

* Given a curvilinear world-line path in spherical coordinates $ds^2 = dr^2 + r\ d \theta^2 + r^2 \sin^2 \theta\ d\phi^2$, we define a metric
    $$ g^{ij} = \begin{pmatrix} 1 & 0 & 0 \\ 0 & r^2 & 0 \\ 0 & 0 & r^2 \sin^2\theta \end{pmatrix}$$

* We also define the Christoffel symbols in summation-notation: $$ \Gamma_{jk}^{i} \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \frac{\partial g^{kl}}{\partial x^j} + \frac{\partial g^{jl}}{\partial x^k} - \frac{\partial g^{jk}}{\partial x^l} \right] $$ or equivalently $$ \Gamma_{jk}^i \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \partial_j g^{kl} + \partial_k g^{jl} - \partial_l g^{jk} \right] $$

* **QUESTION :>** in the definitions above, are the coordinate indices supposed to be upper or lower? Is the definition valid for the contravariant, or the covariant, expression of the indices? Differing definitions across sources. 

In [16]:
# For our variable names, we'll use Unicode U+2054 (CHARACTER TIE) to 
#  indicate contravariant (upper) indices, followed by an underscore preceding
#  covariant indices.
g⁀ij = M.metric()
g⁀ij[1,1], g⁀ij[2,2], g⁀ij[3,3] = 1, r^2, r^2 * (sin(θ))^2
g⁀ij.display()

g = e^r⊗e^r + r^2 e^θ⊗e^θ + r^2*sin(θ)^2 e^φ⊗e^φ

In [17]:
g⁀ij[:]

[           1            0            0]
[           0          r^2            0]
[           0            0 r^2*sin(θ)^2]

In [18]:
# check that the metric has non-zero curvature, unlike the Minkowski metric
g⁀ij.riemann().display()

Riem(g) = -2 e_r⊗e^θ⊗e^r⊗e^θ + 2 e_r⊗e^θ⊗e^θ⊗e^r - 2*sin(θ)^2 e_r⊗e^φ⊗e^r⊗e^φ + 2*sin(θ)^2 e_r⊗e^φ⊗e^φ⊗e^r + 2/r^2 e_θ⊗e^r⊗e^r⊗e^θ - 2/r^2 e_θ⊗e^r⊗e^θ⊗e^r - 2*(2*(r^2 - 1)*sin(θ)^2 + 1)/r^2 e_θ⊗e^φ⊗e^θ⊗e^φ + 2*(2*(r^2 - 1)*sin(θ)^2 + 1)/r^2 e_θ⊗e^φ⊗e^φ⊗e^θ + 2/r^2 e_φ⊗e^r⊗e^r⊗e^φ - 2/r^2 e_φ⊗e^r⊗e^φ⊗e^r + 2*(2*(r^2 - 1)*sin(θ)^2 + 1)/(r^2*sin(θ)^2) e_φ⊗e^θ⊗e^θ⊗e^φ - 2*(2*(r^2 - 1)*sin(θ)^2 + 1)/(r^2*sin(θ)^2) e_φ⊗e^θ⊗e^φ⊗e^θ

In [19]:
g_inv = g⁀ij.inverse()
g_inv[:]

[               1                0                0]
[               0           r^(-2)                0]
[               0                0 1/(r^2*sin(θ)^2)]

In [20]:
from sage.tensor.modules.tensor_with_indices import TensorWithIndices

g_inv = TensorWithIndices(g_inv, '^ij')
g_inv['ij'][:]

[               1                0                0]
[               0           r^(-2)                0]
[               0                0 1/(r^2*sin(θ)^2)]

We are now able to phrase the Christoffel symbols as contractions on the indices, following the definition.  It turns out that Sage Math incorporates 

In [41]:
g⁀ij.christoffel_symbols().display('\Gamma')

\Gamma_122 = -2*r^3 
\Gamma_133 = -2*r^3*sin(θ)^4 
\Gamma_212 = 2/r 
\Gamma_221 = 2/r 
\Gamma_233 = -2*cos(θ)*sin(θ)^3 
\Gamma_313 = 2/r 
\Gamma_323 = 2*cos(θ)/sin(θ) 
\Gamma_331 = 2/r 
\Gamma_332 = 2*cos(θ)/sin(θ)

In [ ]:
from sage.manifolds.operators import grad

christoffel = (1/2) * g_inv['^il'] * ( g⁀ij['kl'] + g⁀ij['jl'] - g⁀ij['jk'] )

christoffel[:]

* recreate the same thing, but use the manifold definition so I can use the tensor transform to calculate the christoffel symbols from that

In [ ]:
#c, t, r, θ, φ = var('c, t, r, θ, φ')

# For convenience, I've copied over the unicode characters here, e.g.: g⸢ij⸣⸤kl⸥ 
M = Manifold(4, "g", latex_name=r'g^{ij}', structure='Lorentzian', field='real') # real, differentiable, manifold with metric g
#M.set_calculus_method('sympy') # use sympy for example, instead of Symbolic Ring


In [ ]:
# switch to spherical coordinate frame
polar.<t, r, θ, φ> = M.chart()
polar.coord_range()

* use the spherical coordinates of the manifold defined for the rank (0,4) tensor to calculate non-zero christoffel symbols using numpy einsum. 